# <font size=7>CF Recommender System</font>

<font size=3.5>Two major kinds of recommender systems:
- **Content-Based**: items are recommended by their intrinsic similarity (clustering).
- **Collaborative Filtering**: items are recommended based on the ratings of users.

The main benefit of CF recommender system is it only need the ratings from users. We do not need to add many tags to every item. It vastly save human labors and time.
Two main CF methods:
- **Neighborhood Method**
- **Latent Factor Method**

<font size=3.5>There is no absolute rank between two methods. In this movie recommender system project, I **only use Neighborhood Method**.
<br\>
    <br\>
The core idea of Neighborhood Method is to remove users’ bias and movies’ popularity. It is well acknowledged that some users are more generous than others and always give ratings higher than average rating while some may always give lower ratings since they are more critical. For the same reason, a movie may have higher or lower rating due to its release time. 

## <font size=5>Algorithm Part</font>

### 1. Data Preprocessing
<br\>
<font size=3.5>Initially, suppose we have a big matrix $R$ consisted with existed ratings, which $r(i, j)$ is the rating that user $u_i$ gives movie $m_j$. In order to make rating $r(i, j)$ be more informative, replacing every $r(i, j)$ in $R$ by the values $r^{*}(i, j) = r(i, j) - \bar{r}$, where $\bar{r}$ means average rating over all existed ratings.
<br\>
    <br\>
Now, the rating is more informative, if rating $r^{*}(i, j) < 0$, it means user  $u_i$ like movie $m_j$ more than average. However, we do not remove personal bias and movie popularity. We need to remove these biases and then we will get a more accurate rating matrix.
<br\>
    <br\>
Therefore, we introduce for every user $u_i$ a variable $\nu_{i}$ standing for personal bias and for every movie $m_j$ a variable $\mu_{j}$ standing for its popularity. Generally, we got two 1D vectors $\vec{\nu} and  \vec{\mu}$. Then, in order to get the best vectors, we should minimize <font size=4>$$Loss(\vec{\nu}, \vec{\mu}) = \sum_{(i, j) \in R}( r^{*}(i, j) - \nu_{i} - \mu_{j})^{2}$$</font>

In [1]:
def convert_to_matrix(df):
    nusers = config.nusers
    nmovies = config.nmovies
    ratings = np.zeros(shape=(nusers,nmovies))
    true_value = np.zeros(shape=(nusers,nmovies))
    avg = df["rating"].mean()
    for i in range(len(df)):
        userID = int(df.loc[i][0]-1)
        movieID = int(df.loc[i][1]-1)
        rating = float(df.loc[i][2])
        ## Get a more informative dataset
        ratings[userID, movieID] = rating - avg
        true_value[userID, movieID] = 1
    return ratings, true_value

### 2. Learning
<br\>
<font size=3.5>This **Least Squares Problem** could be easily solved by **Gradient Descent**: <font size=4>$$\frac{\partial Loss(\vec{\nu}, \vec{\mu})}{\partial \mu_{j}} = \frac{\partial \sum_{(i, j) \in R}( r^{*}(i, j) - \nu_{i} - \mu_{j})^{2}}{\partial \mu_{j}} = -2 \sum_{i : (i, j) \in R}( r^{*}(i, j) - \nu_{i} - \mu_{j})$$ $$\frac{\partial Loss(\vec{\nu}, \vec{\mu})}{\partial \nu_{i}} = \frac{\partial \sum_{(i, j) \in R}( r^{*}(i, j) - \nu_{i} - \mu_{j})^{2}}{\partial \nu_{i}} = -2 \sum_{j : (i, j) \in R}( r^{*}(i, j) - \nu_{i} - \mu_{j})$$</font> Typically, in order to avoid overfitting in least squares problem, introduce **L2 Regularization**: <font size=4>$$Loss(\vec{\nu}, \vec{\mu}) = \sum_{(i, j) \in R}( r^{*}(i, j) - \nu_{i} - \mu_{j})^{2} + \lambda (\sum_{i}\nu_i^{2} + \sum_{i}\mu_j^{2})$$</font>

In [2]:
def learning(learning_rate, epoch, ratings, true_value):
    ## Initialization of user bias vector and movie bias vector
    V = np.zeros(ratings.shape[0])
    U = np.zeros(ratings.shape[1])
    for _ in range(1, epoch+1):
        F = np.add.outer(V,U)
        ## Solve this problem by Gradient Descent with L2 Regularization
        gradient_V = np.sum(ratings-F*true_value, axis = 1) - 2*config.lamda * V
        gradient_U = np.sum(ratings-F*true_value, axis = 0) - 2*config.lamda * U
        V += 2*learning_rate*gradient_V
        U += 2*learning_rate*gradient_U
        if _%100 == 0:
            print(f"{_//100}/{epoch//100} has finished")
            print("Current Loss is: ", np.sum((ratings-F*true_value)**2, axis = (0,1)))
    bias_save("userID", V)
    bias_save("movieID", U)
    return V, U

<font size=3.5>After get the vectors of $\vec{\nu}$ and $\vec{\mu}$. We could get $\tilde{r}(i, j) = r^{*}(i, j) - \nu_{i} - \mu_{j}$ and update the matrix from $R^{*}$ to $\tilde{R}$.
<br\>
    <br\>
Now, we are ready to compute the similarity between users or movies. 
- **Cosine Similarity**: only consider the direction of two vectors.$$cos(\vec{u_i}, \vec{u_k}) = \frac{u_i^\top u_k}{\sqrt{(u_i^\top u_i)}\sqrt{(u_k^\top u_k)}} $$

In [3]:
def similarity(u_i, u_k):
    # Cosine Similarity
    return np.dot(u_i, u_k)/(np.sqrt(np.dot(u_i, u_i))*np.sqrt(np.dot(u_k, u_k)))

<font size=3.5>Note that we are not only interested in the most **similar** users but also the most **dissimilar** users. Therefore, we should consider the top-k users with largest similarity $|sim(U_i, U_k)|$. There are many method to get top-k result fast, such as binary heap, deterministic select, quick select, here I use quick select.

### 3. Prediction
<br\>
<font size=3.5>After we did all of above preprocessing, we could predict a movie for a particular user. The core idea here is to find the movie which most similar users like best or the movie which most dissimilar users dislike. In addition, do not forget to add the personal bias and popularity:$$pred(i, j) = \nu_{i} + \mu_{j} + \frac{\sum_{k \in top-k}sim(U_i, U_k)\tilde{r}(j, k)}{\sum_{k \in top-k}|sim(U_i, U_k)|}$$ Definitely, we will recommend the movie with highest prediction score.

In [4]:
def pred_rating(userID, movieID):
    values = np.zeros(config.nusers)
    u_idx = np.nonzero(ratings[:,movieID-1])[0] # Users who saw movie
    for u in u_idx:
        if u == userID:
            continue
        values[u] = similarity(ratings[userID-1], ratings[u])
    Q = TopK(values, config.MaxUserNumber) # Fast get results and their indices by using quick select
    sim, idx = Q.answer()
    if sum(abs(sim)) == 0: # Avoid zero division
        return 0
    ## Note that we do not only consider most similar but also most dissimilar
    return V[userID-1] + U[movieID-1] + sum([sim[i]*ratings[idx[i],movieID-1] for i in range(config.MaxUserNumber)])/sum(abs(sim))

def prediction(userID):
    m_idx = np.where(ratings[userID-1]==0)[0] # Movies which user did not see yet
    values = np.zeros(config.nmovies)
    for m in m_idx:
        values[m] = pred_rating(userID, m+1)
    return np.argmax(values)+1 # Return index of movieID with highest score